## A small ChatGPT style Transformer

* concepts in NLP and Transformers

* generative mdoels


In [1]:

import torch
import numpy as np
import requests
## import tiktoken
import torch.nn as nn

from torch.nn import functional as F



In [2]:
## !pip install requests
## !pip install tiktoken    ## requires python   >    3.9

In [3]:

torch.manual_seed(1337)


In [4]:

block_size = 8   ## max content length for predictions
batch_size = 4 


In [5]:

input_file_path = 'input.txt'


data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'

with open(input_file_path, 'w') as f:
        f.write(requests.get(data_url).text)
        

In [6]:

with open(input_file_path, 'r') as f:
    text = f.read()
    



In [7]:
print("length of data in characters")
len(text)

length of data in characters


1115394

In [8]:
## n = len(data)


In [9]:
print(  text[:1000]   )

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.




## get list of unique characters


In [10]:

chars = sorted(     list(set(text))   )
chars


['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [11]:

vocab_size = len(chars)
vocab_size 


65

In [12]:

print(  ''.join(chars)  )



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


## tokenizer

create a mapping from characters to integers

* other better options are tiptoken and SentencePiece


In [13]:

stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }

encode = lambda s: [ stoi[c]          for c in s   ]    ## encoder: string to integer
decode = lambda l: ''.join(   itos[i] for i in l   )    ## decoder: interger to string


## Encode and decode

In [14]:

print(   encode("hii there")    )


[46, 47, 47, 1, 58, 46, 43, 56, 43]


In [15]:

print(   decode(   encode("hii there")  )   )


hii there



## Encode the text


In [16]:

data = torch.tensor(   encode(text), dtype=torch.long   )
print(data.shape, data.type)


torch.Size([1115394]) <built-in method type of Tensor object at 0x7fd278037470>


In [17]:
data

tensor([18, 47, 56,  ..., 45,  8,  0])

In [18]:
print(   data[:1000]   )

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 


## Train and test split


In [19]:

n = int(   0.9*len(data)   )
n


1003854

In [20]:

train_data = data[:n]
val_data   = data[n:]



## sample random chunks of block size


In [21]:

train_data[:block_size+1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


## offset by one


In [22]:

x = train_data[:block_size]
y = train_data[1:block_size+1]


In [23]:
x

tensor([18, 47, 56, 57, 58,  1, 15, 47])

In [24]:
y

tensor([47, 56, 57, 58,  1, 15, 47, 58])

In [25]:

for t in range(block_size):
    context = x[: t+1]
    target  = y[t]
    print(f"when input is {context} then the target is: {target}")


when input is tensor([18]) then the target is: 47
when input is tensor([18, 47]) then the target is: 56
when input is tensor([18, 47, 56]) then the target is: 57
when input is tensor([18, 47, 56, 57]) then the target is: 58
when input is tensor([18, 47, 56, 57, 58]) then the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) then the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) then the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) then the target is: 58



## batch processing


In [26]:
len(data)

1115394

In [27]:
block_size

8

In [28]:
len(data) - block_size

1115386

In [29]:
(batch_size,) 

(4,)

In [30]:
ix = torch.randint(   len(data) - block_size, (batch_size,)   )
ix

tensor([1078327,  453969,   41646,  671252])


## Randomly sample batch size (e.g. 4) ids and get the next block_size ids (e.g. 7) for each respectively


In [31]:

def get_batch(split):
    if split == "train":
        data = train_data
    else:
        data = val_data
    ix = torch.randint(   len(data) - block_size, (batch_size,)   )
    x  = torch.stack(    [  data[ i : i+block_size ]   for i in ix]    ) 
    y  = torch.stack(    [  data[ i+1 : i+1+block_size ]   for i in ix]    )
    
    return x, y


In [32]:

xb, yb = get_batch("train")


In [33]:
xb

tensor([[57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46]])

In [34]:
yb

tensor([[43, 60, 43, 52,  1, 63, 43, 39],
        [43, 42,  8,  0, 25, 63,  1, 45],
        [42,  5, 57,  1, 57, 39, 49, 43],
        [57, 58, 63,  6,  1, 58, 46, 47]])

In [35]:

for b in range(batch_size):         
    for t in range(block_size):
        context = xb[b, :t+1]
        target  = yb[b, t]
        print(f"when input is {context.tolist()} the target is: {target}")


when input is [57] the target is: 43
when input is [57, 43] the target is: 60
when input is [57, 43, 60] the target is: 43
when input is [57, 43, 60, 43] the target is: 52
when input is [57, 43, 60, 43, 52] the target is: 1
when input is [57, 43, 60, 43, 52, 1] the target is: 63
when input is [57, 43, 60, 43, 52, 1, 63] the target is: 43
when input is [57, 43, 60, 43, 52, 1, 63, 43] the target is: 39
when input is [60] the target is: 43
when input is [60, 43] the target is: 42
when input is [60, 43, 42] the target is: 8
when input is [60, 43, 42, 8] the target is: 0
when input is [60, 43, 42, 8, 0] the target is: 25
when input is [60, 43, 42, 8, 0, 25] the target is: 63
when input is [60, 43, 42, 8, 0, 25, 63] the target is: 1
when input is [60, 43, 42, 8, 0, 25, 63, 1] the target is: 45
when input is [56] the target is: 42
when input is [56, 42] the target is: 5
when input is [56, 42, 5] the target is: 57
when input is [56, 42, 5, 57] the target is: 1
when input is [56, 42, 5, 57, 1] 


## Bi-gram model


In [36]:

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        
        logits = self.token_embedding_table(idx)      ## batch, time, vocab_size (4, 8, 65)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits  = logits.view(B*T, C)
            targets  = targets.view(B*T)
            loss   = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        ## idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            
            logits, loss = self(idx)
            ## focus only on last time stamp
            logits = logits[:, -1, :]           ## becomes (B, C)
            probs = F.softmax(logits, dim= -1)    ## (B, C)
            idx_next = torch.multinomial(probs, num_samples=1)     ## (B, 1)
            ## append sample to the running sequence
            idx = torch.cat(  (idx, idx_next), dim=1  )            ## (B, T+1)
        return idx
            
            
            
            

In [37]:

m   = BigramLanguageModel(vocab_size)

logits, loss = m(xb, yb)

print(logits.shape)



torch.Size([32, 65])


In [38]:
print(loss)

tensor(4.8549, grad_fn=<NllLossBackward0>)


In [39]:
logits

tensor([[ 1.1564, -0.7436, -0.3880,  ...,  0.9058,  0.5203,  0.3977],
        [-0.8341, -1.2283, -1.3155,  ..., -2.2339, -1.4059,  1.2754],
        [ 2.7071,  0.7921,  0.9866,  ...,  0.0660,  0.3611, -0.1090],
        ...,
        [-1.9560, -0.8003, -0.5045,  ..., -1.5630,  1.1346, -0.0824],
        [-0.2622,  1.7675, -0.6312,  ...,  0.3641, -0.8859, -0.6271],
        [-0.3642, -0.1543, -1.1895,  ..., -0.1503, -0.1571, -0.4342]],
       grad_fn=<ViewBackward0>)


## Now generate random text before training the BiGram model


In [40]:

## Kick off generation with some starting token. In this case id 0

idx = torch.zeros(  (1, 1),  dtype=torch.long   )

gen_text = m.generate(idx, max_new_tokens=100)[0].tolist()

print(  decode(gen_text)  )



VMPFsPHSYAHIkPXroxdH.ACNM?''sBym,SG?Um-FhLZkRNsg$PrIPyXMDqA'XH.VsaT.fKxhQCWWsjkHMhsRSUaN$j?WTcK3MHMF



## Training and Optimization


In [41]:

optimizer = torch.optim.Adam(  m.parameters(), lr=1e-3   )


In [42]:

batch_size = 32

for step in range(10000):
    xb, yb = get_batch('train')
    
    ## evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)   ## zero out
    loss.backward()
    optimizer.step()
    
    if step % 1000 == 0:
        print(loss.item())


4.757808685302734
3.7833778858184814
3.201312303543091
2.9419002532958984
2.7246687412261963
2.4351015090942383
2.4883556365966797
2.511401891708374
2.4464640617370605
2.4443509578704834


## now, regenerate after some training

In [43]:

## Kick off generation with some starting token. In this case id 0

idx = torch.zeros(  (1, 1),  dtype=torch.long   )

gen_text = m.generate(idx, max_new_tokens=500)[0].tolist()

print(  decode(gen_text)  )



hot spit r
DWher, mbeicou tis, iss, be atnoorerst, byou,

G t to thealourds shal-wndy, Gor mangasosced, s r ghoshaveeavHer toiknor cou bomithemiutho lelor or s.
INourdito my theais tervet thiule d har s h s? o hee,
A minoverqube wisuby le tonda apawakseey,'s jestw?
Yor, shang I benan thiscouom four
LUShathrithendat, Le' vie p'r ttr mbes tir theny s wa ERINI son s aterondes orim ag t cers grthat g t.
FozNI ainouriau makes;
Cors fis d hengspleatan JOLEYo goforsutouterens be this y s tawethe se st:



## The Mathematical Trick in Self-Attention


In [44]:

torch.manual_seed(1337)
B, T, C = 4, 8, 2            ## batch, time, channels
x = torch.randn( B, T, C )
x.shape


torch.Size([4, 8, 2])


Now we just look at average of all previous tokens up to token t. We do not look at future tokens. 


In [45]:

## we want x[b, t] = mean_{i<=t} x[b, i]

xbow = torch.zeros(  (B, T, C)  )
for b in range(B):
    for t in range(T):
        xprev      = x[b, :t+1]        ## (t, C)
        xbow[b, t] = torch.mean(  xprev, 0  )


In [46]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [47]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


## Example

instead of for loops, we use matrix multiplications


In [48]:

torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print('a=')
print(a)
print('---------')
print('b=')
print(b)
print('---------')
print('dot_product ->> c=')
print(c)
print('---------')


a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
---------
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---------
dot_product ->> c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])
---------



## Masking 


In [49]:


torch.manual_seed(42)
a = torch.tril(  torch.ones(3, 3)    )
a = a /torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print('a=')
print(a)
print('---------')
print('b=')
print(b)
print('---------')
print('dot_product ->> c=')
print(c)
print('---------')


a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---------
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---------
dot_product ->> c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
---------


In [50]:


wei = torch.tril(  torch.ones(T, T)   )
wei = wei / wei.sum(1, keepdim = True)
wei


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [51]:

xbow2 = wei @ x        ## (B, T, T) @ (B, T, C)   ---->>>   (B, T, C)


In [52]:

torch.allclose(   xbow,  xbow2   )        ## this compares that they are the same


True

## Re-written in a third way (using softmax)

* use this for self-attention 

* weighted aggreagation of tokens


In [53]:

tril = torch.tril(    torch.ones(T, T)    )
wei  = torch.zeros(   (T, T)   )
wei  = wei.masked_fill(tril == 0, float('-inf'))
wei  = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(  xbow, xbow3    )


True

## Self attention for one head

In [58]:

torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

## a single head with self-attention
head_size = 16
key   = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)           # (B, T, 16)
q = query(x)         # (B, T, 16)

## Because of batches, only transpose the last 2 dimensions
wei = q @ k.transpose(-2, -1)        ## (B, T, 16) @ (B, 16, T)  --> (B, T, T) 

tril = torch.tril(    torch.ones(T, T)    )
wei  = wei.masked_fill(tril == 0, float('-inf'))
wei  = F.softmax(wei, dim = -1)

v = value(x)
out = wei @ v

out.shape


torch.Size([4, 8, 16])

In [59]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089